In [ ]:
# processes historical SPEI data

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import os

In [2]:
data_folder = Path(r'F:\World Bank\City Scan\data\SPEI')  # change file path as needed

In [29]:
periods = ['01', '12', '48']
years = range(2011, 2021)

In [4]:
centroids = pd.read_csv('centroids.csv')

In [5]:
centroids

,city,x,y,utm
0,Kinshasa,15.352551,-4.410595,32633
1,Kananga,22.419693,-5.893789,32634
2,Mbuji-Mayi,23.612801,-6.132235,32634
3,Lubumbashi,27.488133,-11.650550,32635
4,Kisangani,25.191068,0.523556,32635
5,Bukavu,28.847353,-2.518455,32635
6,Goma,29.198201,-1.639446,32635
7,Tshikapa,20.781687,-6.425809,32634
8,Mwene-Ditu,23.453712,-7.006963,32634
9,Gemena,19.789188,3.247033,32634


In [72]:
spei_val = dict({p: dict({c: {} for c in centroids.city}) for p in periods})

In [73]:
for period in periods:
    spei_nc = xr.open_dataset(data_folder / ('spei'+period+'.nc'))

    for index, row in centroids.iterrows():
        for year in years:
            for month in range(1, 13):
                time1 = str(year) + '-' + str(month) + '-15'
                val = spei_nc.sel(lon = row['x'], lat = row['y'], time = time1, method = 'nearest')['spei'].to_dict()['data']
                spei_val[period][row['city']][time1] = val

In [76]:
# write to csv
for period in periods:
    with open('stats/spei/spei'+period+'.csv', 'w') as f:
        f.write('city,date,spei\n')
        for city in centroids.city:
            for year in years:
                for month in range(1, 13):
                    time1 = str(year) + '-' + str(month) + '-15'
                    f.write('%s,%s,%s\n' % (city, time1, spei_val[period][city][time1]))